<a href="https://colab.research.google.com/github/tbazzi/Data-Science---Fullstack-Bootcamp/blob/master/S9_4_MLlib_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Chargement des données

In [0]:
spark.conf.set(
  "fs.azure.account.key.storagestudent.blob.core.windows.net", 
  "pH3rgal+XcwJXc3hQEYEAE+dMBo6YzhKnb4iYQNlTZ9lXaxe8RWmZwVPMF1j2V5zwBnBZ/iNu8JoFgApOxdn4Q=="
)

salary = spark.read.load("wasbs://default@storagestudent.blob.core.windows.net/datasets/S8-4/Cours/salary.parquet", format="parquet")

#1. Pre processing

In [0]:
from pyspark.ml.feature import StringIndexer, IndexToString

label_indexer = StringIndexer(inputCol="salary", outputCol="label")

In [0]:
display(salary)

age,workclass,education,marital_status,occupation,relationship,race,sex,native_country,hours_per_week,salary
39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,40,<=50K
50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,13,<=50K
38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,40,<=50K
53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,40,<=50K
28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,40,<=50K
37,Private,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States,40,<=50K
49,Private,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,Jamaica,16,<=50K
52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,45,>50K
31,Private,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,United-States,50,>50K
42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,40,>50K


In [0]:
from pyspark.ml.feature import StringIndexer, IndexToString

label_indexer = StringIndexer(inputCol="salary", outputCol="label")

string_columns = ["workclass",  "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]
string_indexers = [StringIndexer(inputCol=col, outputCol=col+"Index") for col in string_columns]

from pyspark.ml.feature import OneHotEncoderEstimator

one_hot_encoder = OneHotEncoderEstimator(
  inputCols=[col+"Index" for col in string_columns],
  outputCols=[col+"Vec" for col in string_columns]
)

from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler(
    inputCols=[col+"Vec" for col in string_columns] + ["age", "hours_per_week"],
    outputCol="features"
)

from pyspark.ml import Pipeline

preprocessing_pipeline = Pipeline(
  stages= [label_indexer, *string_indexers, one_hot_encoder, vector_assembler]
)

In [0]:
preprocessing_pipeline_fitted = preprocessing_pipeline.fit(salary)

salary_preprocessed = preprocessing_pipeline_fitted.transform(salary)

In [0]:
display(salary_preprocessed.select("label", "features"))

label,features
0.0,"List(0, 96, List(4, 10, 24, 32, 44, 48, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 39.0, 40.0))"
0.0,"List(0, 96, List(1, 10, 23, 31, 43, 48, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 50.0, 13.0))"
0.0,"List(0, 96, List(0, 8, 25, 38, 44, 48, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 38.0, 40.0))"
0.0,"List(0, 96, List(0, 13, 23, 38, 43, 49, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 53.0, 40.0))"
0.0,"List(0, 96, List(0, 10, 23, 29, 47, 49, 62, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 28.0, 40.0))"
0.0,"List(0, 96, List(0, 11, 23, 31, 47, 48, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 37.0, 40.0))"
0.0,"List(0, 96, List(0, 18, 28, 34, 44, 49, 66, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 49.0, 16.0))"
1.0,"List(0, 96, List(1, 8, 23, 31, 43, 48, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 52.0, 45.0))"
1.0,"List(0, 96, List(0, 11, 24, 29, 44, 48, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 31.0, 50.0))"
1.0,"List(0, 96, List(0, 10, 23, 31, 43, 48, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 42.0, 40.0))"


#2. Entrainement du modèle

In [0]:
salary_train, salary_test = salary_preprocessed.randomSplit([0.7, 0.3], seed=100)

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label", featuresCol="features")

In [0]:
lr_fitted = lr.fit(salary_train)

In [0]:
predictions_train = lr_fitted.transform(salary_train)
predictions_test = lr_fitted.transform(salary_test)

In [0]:
display(predictions_test)

age,workclass,education,marital_status,occupation,relationship,race,sex,native_country,hours_per_week,salary,label,workclassIndex,educationIndex,marital_statusIndex,occupationIndex,relationshipIndex,raceIndex,sexIndex,native_countryIndex,raceVec,occupationVec,marital_statusVec,educationVec,relationshipVec,sexVec,native_countryVec,workclassVec,features,rawPrediction,probability,prediction
17,?,10th,Never-married,?,Own-child,Other,Female,United-States,20,<=50K,0.0,3.0,7.0,1.0,7.0,2.0,4.0,1.0,0.0,"List(0, 4, List(), List())","List(0, 14, List(7), List(1.0))","List(0, 6, List(1), List(1.0))","List(0, 15, List(7), List(1.0))","List(0, 5, List(2), List(1.0))","List(0, 1, List(), List())","List(0, 41, List(0), List(1.0))","List(0, 8, List(3), List(1.0))","List(0, 96, List(3, 15, 24, 36, 45, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 20.0))","List(1, 2, List(), List(7.500257320358823, -7.500257320358823))","List(1, 2, List(), List(0.9994473635073768, 5.526364926231683E-4))",0.0
17,?,10th,Never-married,?,Own-child,White,Female,United-States,15,<=50K,0.0,3.0,7.0,1.0,7.0,2.0,0.0,1.0,0.0,"List(0, 4, List(0), List(1.0))","List(0, 14, List(7), List(1.0))","List(0, 6, List(1), List(1.0))","List(0, 15, List(7), List(1.0))","List(0, 5, List(2), List(1.0))","List(0, 1, List(), List())","List(0, 41, List(0), List(1.0))","List(0, 8, List(3), List(1.0))","List(0, 96, List(3, 15, 24, 36, 45, 48, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 15.0))","List(1, 2, List(), List(7.666820604348002, -7.666820604348002))","List(1, 2, List(), List(0.9995321152712942, 4.678847287059137E-4))",0.0
17,?,10th,Never-married,?,Own-child,White,Female,United-States,17,<=50K,0.0,3.0,7.0,1.0,7.0,2.0,0.0,1.0,0.0,"List(0, 4, List(0), List(1.0))","List(0, 14, List(7), List(1.0))","List(0, 6, List(1), List(1.0))","List(0, 15, List(7), List(1.0))","List(0, 5, List(2), List(1.0))","List(0, 1, List(), List())","List(0, 41, List(0), List(1.0))","List(0, 8, List(3), List(1.0))","List(0, 96, List(3, 15, 24, 36, 45, 48, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 17.0))","List(1, 2, List(), List(7.608109042175227, -7.608109042175227))","List(1, 2, List(), List(0.9995038366390274, 4.961633609725132E-4))",0.0
17,?,10th,Never-married,?,Own-child,White,Female,United-States,20,<=50K,0.0,3.0,7.0,1.0,7.0,2.0,0.0,1.0,0.0,"List(0, 4, List(0), List(1.0))","List(0, 14, List(7), List(1.0))","List(0, 6, List(1), List(1.0))","List(0, 15, List(7), List(1.0))","List(0, 5, List(2), List(1.0))","List(0, 1, List(), List())","List(0, 41, List(0), List(1.0))","List(0, 8, List(3), List(1.0))","List(0, 96, List(3, 15, 24, 36, 45, 48, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 20.0))","List(1, 2, List(), List(7.520041698916063, -7.520041698916063))","List(1, 2, List(), List(0.9994581837640015, 5.418162359985624E-4))",0.0
17,?,10th,Never-married,?,Own-child,White,Male,United-States,8,<=50K,0.0,3.0,7.0,1.0,7.0,2.0,0.0,0.0,0.0,"List(0, 4, List(0), List(1.0))","List(0, 14, List(7), List(1.0))","List(0, 6, List(1), List(1.0))","List(0, 15, List(7), List(1.0))","List(0, 5, List(2), List(1.0))","List(0, 1, List(0), List(1.0))","List(0, 41, List(0), List(1.0))","List(0, 8, List(3), List(1.0))","List(0, 96, List(3, 15, 24, 36, 45, 48, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 8.0))","List(1, 2, List(), List(7.059821518153839, -7.059821518153839))","List(1, 2, List(), List(0.999141805746025, 8.581942539750565E-4))",0.0
17,?,10th,Never-married,?,Own-child,White,Male,United-States,25,<=50K,0.0,3.0,7.0,1.0,7.0,2.0,0.0,0.0,0.0,"List(0, 4, List(0), List(1.0))","List(0, 14, List(7), List(1.0))","List(0, 6, List(1), List(1.0))","List(0, 15, List(7), List(1.0))","List(0, 5, List(2), List(1.0))","List(0, 1, List(0), List(1.0))","List(0, 41, List(0), List(1.0))","List(0, 8, List(3), List(1.0))","List(0, 96, List(3, 15, 24, 36, 45, 48, 52, 53, 94, 95), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 17.0, 25.0))","List(1, 2, List(), List(6.5607732

In [0]:
display(predictions_test.select("salary", "label", "prediction", "probability"))

salary,label,prediction,probability
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9994473635073768, 5.526364926231683E-4))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9995321152712942, 4.678847287059137E-4))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9995038366390274, 4.961633609725132E-4))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9994581837640015, 5.418162359985624E-4))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.999141805746025, 8.581942539750565E-4))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9985872074920739, 0.0014127925079260707))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9983642133523223, 0.0016357866476777205))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9983642133523223, 0.0016357866476777205))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9978073219073987, 0.0021926780926011767))"
<=50K,0.0,0.0,"List(1, 2, List(), List(0.9978073219073987, 0.0021926780926011767))"


#3. Evaluation et analyses

In [0]:
print("Coefficients: ", lr_fitted.coefficients)
print("Intercepts: ", lr_fitted.intercept)

Coefficients: [-0.786074109022,-1.36258302031,-0.912788091584,-0.799508680098,-1.14328508721,-0.531279849467,-0.356174172559,-1.26105988969,0.762729351606,1.09716305261,1.9397243061,2.25882792201,1.27969409232,0.0511567941814,1.40772116946,0.0206354596896,-0.641448532094,2.92718363032,-0.256450413778,0.260504064311,2.87897070645,-0.156036368407,-0.36358481457,-0.338167242666,-3.17432783069,-2.78384631915,-2.81681355061,-2.65421112557,-2.69568731736,0.218198983257,-0.278153232627,0.482874999831,-0.31502526752,-0.0812832938304,-1.2711803038,-0.749493786003,-1.07955284158,-0.499508068635,-1.07766499226,-1.27183692234,0.222688059061,0.0919107208157,-3.39250366702,0.292718251207,1.08786580151,0.0687957136243,0.898114799968,1.5053379418,-0.0197843785572,-0.197527833678,0.181810716947,-0.358680764039,0.812489553799,-1.98570490629,-2.83740175886,-2.19151596649,-2.13798102453,-2.11092495162,-2.18606737896,-1.69416199832,-2.94506605656,-2.59449375742,-2.01155296085,-1.85921300789,-3.10805042827,-3.09114319038,-2.04989253511,-1.33851555513,-3.12785511145,-2.45424604808,-3.74318221252,-2.20810376001,-3.09531530724,-4.27997408276,-2.45991863844,-1.76172916754,-2.50127000718,-2.1329866288,-2.73689179831,-2.24059951681,-2.94751977707,-2.74196060102,-1.87065046446,-0.981353537115,-2.76720067657,-2.92929161773,-1.20372973469,-3.1698019318,-3.06850070748,-1.98286411949,-10.8506226337,-2.92594365691,-2.17830932639,-2.24032958036,0.027866914976,0.0293557810864]
Intercepts: -1.6114474113302855

In [0]:
from itertools import chain
import pandas as pd

pd.DataFrame(sorted([
  (lr_fitted.coefficients[attr["idx"]], attr["name"]) 
  for attr in chain( *predictions_test.schema["features"].metadata["ml_attr"]["attrs"].values() )
], reverse=True), columns=["Coeff", "Features"])

Out[ 13 ]: 
 Coeff Features
0 2.927184 educationVec_ Prof-school
1 2.878971 educationVec_ Doctorate
2 2.258828 educationVec_ Masters
3 1.939724 educationVec_ Bachelors
4 1.505338 relationshipVec_ Wife
5 1.407721 educationVec_ Assoc-acdm
6 1.279694 educationVec_ Assoc-voc
7 1.097163 educationVec_ Some-college
8 1.087866 relationshipVec_ Not-in-family
9 0.898115 relationshipVec_ Unmarried
10 0.812490 sexVec_ Male
11 0.762729 educationVec_ HS-grad
12 0.482875 occupationVec_ Exec-managerial
13 0.292718 relationshipVec_ Husband
14 0.260504 educationVec_ 12th
15 0.222688 occupationVec_ Tech-support
16 0.218199 occupationVec_ Prof-specialty
17 0.181811 raceVec_ Asian-Pac-Islander
18 0.091911 occupationVec_ Protective-serv
19 0.068796 relationshipVec_ Own-child
20 0.051157 educationVec_ 11th
21 0.029356 hours_per_week
22 0.027867 age
23 0.020635 educationVec_ 10th
24 -0.019784 raceVec_ White
25 -0.081283 occupationVec_ Sales
26 -0.156036 educationVec_ 5th-6th
27 -0.197528 raceVec_ Black
28 -0.256450 educationVec_ 9th
29 -0.278153 occupationVec_ Craft-repair
.. ... ...
66 -2.208104 native_countryVec_ Poland
67 -2.240330 native_countryVec_ Hungary
68 -2.240600 native_countryVec_ Greece
69 -2.454246 native_countryVec_ Japan
70 -2.459919 native_countryVec_ Haiti
71 -2.501270 native_countryVec_ Taiwan
72 -2.594494 native_countryVec_ India
73 -2.654211 marital_statusVec_ Widowed
74 -2.695687 marital_statusVec_ Married-spouse-absent
75 -2.736892 native_countryVec_ Nicaragua
76 -2.741961 native_countryVec_ Ecuador
77 -2.767201 native_countryVec_ Hong
78 -2.783846 marital_statusVec_ Divorced
79 -2.816814 marital_statusVec_ Separated
80 -2.837402 native_countryVec_ Mexico
81 -2.925944 native_countryVec_ Scotland
82 -2.929292 native_countryVec_ Thailand
83 -2.945066 native_countryVec_ El-Salvador
84 -2.947520 native_countryVec_ Peru
85 -3.068501 native_countryVec_ Laos
86 -3.091143 native_countryVec_ South
87 -3.095315 native_countryVec_ Vietnam
88 -3.108050 native_countryVec_ China
89 -3.127855 native_countryVec_ Dominican-Republic
90 -3.169802 native_countryVec_ Trinadad&Tobago
91 -3.174328 marital_statusVec_ Never-married
92 -3.392504 occupationVec_ Priv-house-serv
93 -3.743182 native_countryVec_ Guatemala
94 -4.279974 native_countryVec_ Columbia
95 -10.850623 native_countryVec_ Outlying-US(Guam-USVI-etc)

[96 rows x 2 columns]

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator_auc = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

evaluation_train = evaluator_auc.evaluate(predictions_train)
evaluation_test = evaluator_auc.evaluate(predictions_test)

print(
  "AUC train : {0} - AUC test : {1}"
  .format(
    evaluator_auc.evaluate(predictions_test), 
    evaluator_auc.evaluate(predictions_test)
  )
)

AUC train : 0.8862630721437379 - AUC test : 0.8862630721437137

In [0]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

cf_count = predictions_test.select(F.col("prediction"), F.col("label").cast(IntegerType()))\
                           .groupBy("prediction")\
                           .pivot("label")\
                           .count() 

display(cf_count)

prediction,0,1
0.0,10234,1580
1.0,845,1953


In [0]:
total_count = predictions_test.count()

cf = cf_count.select(
  F.col("prediction"),
  (F.col("0") / total_count).alias("0"),
  (F.col("1") / total_count).alias("1")
)

display(cf)

prediction,0,1
0.0,0.7003832466465918,0.10813030385984122
1.0,0.05782918149466192,0.133657267998905


#4. Cross validation

In [0]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

#paramGrid = ParamGridBuilder()\
#             .addGrid(lr.regParam, [0.01, 0.5, 2.0])\
#             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
#             .addGrid(lr.maxIter, [1, 5, 10])\
#             .build()

paramGrid = ParamGridBuilder()\
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])\
             .build()
 
cv_lr = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator_auc, numFolds=5)
cv_lr_fitted = cv_lr.fit(salary_test)

In [0]:
predictions_train_after_cv = cv_lr_fitted.transform(salary_train)
predictions_test_after_cv = cv_lr_fitted.transform(salary_test)

print(
  "AUC test : {0} - AUC test : {1}".format(
    evaluator_auc.evaluate(predictions_train_after_cv), 
    evaluator_auc.evaluate(predictions_test_after_cv)
  )
)

AUC test : 0.8862810187898986 - AUC test : 0.8886608224743523

In [0]:
bestModel = cv_lr_fitted.bestModel

bestModelParameters = {
  key.name: value
  for key, value in bestModel.extractParamMap().items()
  if key.name in ["regParam", "elasticNetParam", "maxIter"]
}

print(bestModelParameters)

{'maxIter': 100, 'regParam': 0.01, 'elasticNetParam': 0.0}